<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Inicialização de Bibliotecas, Pastas e Foco da Camera</p>

In [1]:
import cv2
import numpy as np
import os
import time
import random
import data_processing as dp
from collections import deque

os.makedirs("../data", exist_ok=True)
os.makedirs("../data/raw", exist_ok=True)

for i in range(7):
    os.makedirs(f"../data/raw/{i}", exist_ok=True)

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)
cap.set(cv2.CAP_PROP_EXPOSURE, -1)

cap.set(cv2.CAP_PROP_AUTOFOCUS, 0)
cap.set(cv2.CAP_PROP_FOCUS, 10)

print("Foco Ajustado")


center = slice(160,None),slice(160,480)     # 0 
left = slice(240,None),slice(None,160)      # 1
left_up = slice(None,240),slice(None,160)   # 2
up_left = slice(None,160),slice(160,320)    # 3
up_right = slice(None,160),slice(320,480)   # 4
right_up = slice(None,240),slice(480,None)  # 5
right = slice(240,None),slice(480,None)     # 6

regioes = {
    0: [center],
    1: [left],
    2: [left_up],
    3: [up_left, right],
    4: [up_right, left],
    5: [right_up],
    6: [right_up]
}


Foco Ajustado


<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Principal</p>

In [4]:
cap = cv2.VideoCapture(0)
division = 16 # --------> Division
frames_queue = deque(maxlen=10)
position_list = [x for x in range(7)]
select = None

while True:
    if select == None:
            select = random.choice(position_list)

    ret,frame_original = cap.read()

    if not ret:
        break

    frame_original = dp.mirroring(frame_original)
    frame = frame_original.copy()
    frame = dp.gray_scale(frame)
    frame = dp.background_subtraction(cam_frame=frame,start_time=0.1,limiar=30)
    input,frame = dp.down_sampling(frame,division=division)
    frame = dp.binarization(frame,limiar=5)
    input = dp.binarization(input,limiar=5)
    original_view = frame_original.copy()
    bbox_points,original_view = dp.bounding_box(frame,frame_vizualizer=frame_original,division=division)
    input = dp.centralize(input, bbox_points=bbox_points)
    input = dp.dilate(input,kernel=np.ones(shape=(3,3)))
    
    frames_queue.append(input)

    for regiao in regioes.get(select, []):
        original_view[regiao] = [0, 0, 255]

    for regiao in regioes.get(select, []):
        
        if (all(np.sum(frame[regiao]) > 0 for regiao in regioes.get(select, []))) & (len(frames_queue) >= 10):

            folder = os.listdir(fr'../data/raw/{select}')
            if not folder:
                name = 0
            else:
                folder = [int(os.path.splitext(x)[0]) for x in folder]
                name = max(folder) + 1
                
            np.save(fr'../data/raw/{select}/{name}.npy', np.array(frames_queue))
            position_list.remove(select)
            select = None
            break

    if position_list == []:
        break
    print(len(frames_queue))

    if cv2.waitKey(33) & 0xFF == ord('q'):
        break
    
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        dp.background_subtraction.background = dp.gray_scale(frame_original.copy())
        frame = np.full_like(frame_original,255)
        print('Background reset')

    cv2.imshow('frame',frame)
    cv2.imshow('Bounding Box Frame',original_view)
    cv2.imshow('sample',input)

cap.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7


<p style="color: gray; 
          text-align: center;
          font-size: 24px;">
Bloco Video REC
</p>

In [ ]:
gravador = cv2.VideoWriter('../media/.bin/asd.avi',fourcc=cv2.VideoWriter_fourcc(*'XVID'),fps=30,frameSize=(640,480))
cam = cv2.VideoCapture(0)
rec = False
while True:
    ret,original_frame = cam.read()
    original_frame = dp.mirroring(original_frame)

    if not ret:
        break
    
    frame = original_frame.copy()
    frame = dp.gray_scale(frame)
    frame = dp.background_subtraction(cam_frame=frame,start_time=3,limiar=15)
    vector,frame = dp.down_sampling(frame,division=16)
    frame = dp.binarization(frame,limiar=5)
    vector = dp.binarization(vector,limiar=5)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(1) & 0xFF == ord('1'):
        dp.background_subtraction.background = dp.gray_scale(original_frame.copy())
        frame_original = np.full_like(original_frame,255)
    elif cv2.waitKey(1) & 0xFF == ord('2'):
        rec = True
        print('gravandooo...')

    if rec:
        frame = np.stack([frame]*3, axis=-1)
        gravador.write(frame)


    cv2.imshow('Original',original_frame)
    cv2.imshow('Visualizer',frame)

cam.release()
gravador.release()
cv2.destroyAllWindows()